# Assignment of building damage data to grids from municipalities

This notebook that uses the weights from the area and the number of buildings.
The rasterisation of damage data is done in this notebook.

In [104]:
%load_ext jupyter_black
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv

load_dotenv()

pd.set_option("display.float_format", lambda x: "%.5f" % x)
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input_fji/02_housing_damage/input/"
)
baseline_input_dir = (
    Path(os.getenv("STORM_DATA_DIR")) / "analysis/01_baseline_model_fji/input/"
)
output_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input_fji/02_housing_damage/output/"
)

SUVA_PCODE = "FJ41204"

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [94]:
# reading in file with building damage data and adm3 in grid
# don't bother since going by buildings instead
# adm3_perc_ingrid = pd.read_csv(
#     input_dir / "Adm3_Perc_inGrid_Weight_Matrix.csv"
# )
perc_build_dmg_data = pd.read_csv(
    baseline_input_dir / "combined_input_data.csv"
)
build_dmg_data = pd.read_csv(input_dir / "fji_impact_data.csv")
fji_build_weights = pd.read_csv(input_dir / "fji_bld_weight_matrix.csv")
fji_build_grid = pd.read_csv(
    input_dir / "fji_hotosm_bld_centroid_grid_count.csv"
)
fji_build_municip = pd.read_csv(
    input_dir / "fji_hotosm_bld_centroid_municip_count.csv"
)

# distribute damage over adm3s based on number of buildings

build_dmg_data["typhoon"] = build_dmg_data["Name Season"].apply(
    lambda x: x.split(" ")[0]
)

build_dmg_data = build_dmg_data[["typhoon", "Houses Destroyed"]]

df_buildings = fji_build_municip.copy()
df_buildings["frac_buildings"] = (
    df_buildings["numbuildings"] / df_buildings["numbuildings"].sum()
)
df_buildings = df_buildings.set_index("ADM3_PCODE")

for pcode in df_buildings.index:
    build_dmg_data[pcode] = (
        df_buildings.loc[pcode, "frac_buildings"]
        * build_dmg_data["Houses Destroyed"]
    )

build_dmg_data = build_dmg_data.drop(columns=["Houses Destroyed"])
build_dmg_data = build_dmg_data.melt(
    id_vars=["typhoon"], var_name="pcode", value_name="Totally"
)

build_dmg_data = build_dmg_data[build_dmg_data["Totally"] > 0]
build_dmg_data

,typhoon,pcode,Totally
9,Yasa,FJ10201,34.44342
10,Ana,FJ10201,0.49871
11,Winston,FJ10201,488.56247
12,Keni,FJ10201,2.42922
13,Josie,FJ10201,1.04569
...,...,...,...
7795,Tia,FJ41405,0.98583
7798,Mick,FJ41405,1.30130
7809,Gene,FJ41405,0.00986
7818,Wilma,FJ41405,0.23660


In [95]:
build_dmg_data[build_dmg_data["pcode"] == SUVA_PCODE]

,typhoon,pcode,Totally
7107,Yasa,FJ41204,159.60355
7108,Ana,FJ41204,2.31093
7109,Winston,FJ41204,2263.89542
7110,Keni,FJ41204,11.25649
7111,Josie,FJ41204,4.84551
7112,Gita,FJ41204,1.64002
7113,Evan,FJ41204,149.09252
7128,Ami,FJ41204,198.44215
7146,Oscar,FJ41204,175.55645
7149,Arthur,FJ41204,111.81939


We will come back to this 
as it is not clear why there are multiple rows for the same municipalities 
recorded for the same typhoon and Year in the building damage data set. 
Different total values could be accumulations. For now, treating them as separate values.

In [96]:
## aggregating these values
## To confirm whether they are additional data or cumulative data in the case where the total values are different for same typhoons.

# build_dmg_data_grouped = (
#    build_dmg_data.groupby(["pcode", "typhoon", "Year"]).sum().reset_index()
# )
# build_dmg_data_grouped[build_dmg_data_grouped["pcode"] == "PH025012000"]

In [97]:
all(
    item in list(fji_build_municip["ADM3_PCODE"])
    for item in list(build_dmg_data["pcode"])
)
# Not all municipalities are in the damage data set.
# Not all municipalities in the building damage data can be found in the admin 3 shapefile.
# removing those buildings with incorrect pcode in them
build_dmg_data_grouped = build_dmg_data[
    build_dmg_data["pcode"].isin(list(fji_build_municip["ADM3_PCODE"]))
]
build_dmg_data_grouped["Totally"].sum()

47062.0

## Using Area of Municipality

In [98]:
## Section describing the merging of the north and south buildings from HOTOSM shapefile
merged_df_points = fji_build_grid.copy()
merged_df_points = pd.merge(
    merged_df_points,
    fji_build_grid.drop("numbuildings", axis=1),
    on="id",
    suffixes=(None, "_y"),
)

merged_df_points["numbuildings"].describe()

count     380.00000
mean      681.33158
std      1914.96398
min         0.00000
25%        44.00000
50%       202.50000
75%       631.75000
max     25215.00000
Name: numbuildings, dtype: float64

In [99]:
merged_df_points_overlap = merged_df_points.loc[
    merged_df_points["Centroid"].isin(adm3_perc_ingrid["Centroid"])
]

In [100]:
### Section describing the computation of the building damage percentage
# combining building damage data and grid information
merged_total_damage_df = adm3_perc_ingrid.merge(
    build_dmg_data_grouped,
    left_on="ADM3_PCODE",
    right_on="pcode",
    how="left",
)

In [101]:
# computing % in each grid
# totally damaged
merged_total_damage_df["Totally_Damaged_bygrid"] = (
    merged_total_damage_df["Municipality Completeness"]
    * merged_total_damage_df["Totally"]
)

In [105]:
merged_total_damage_df[
    (merged_total_damage_df["pcode"] == SUVA_PCODE)
    & (merged_total_damage_df["typhoon"] == "Yasa")
]

,id,Centroid,ADM3_PCODE,ADM3_EN,Municipality Completeness,typhoon,pcode,Totally,Totally_Damaged_bygrid


In [44]:
test_df = (
    merged_total_damage_df[["ADM3_PCODE", "Totally_Damaged_bygrid"]]
    .groupby("ADM3_PCODE")
    .sum()
    .reset_index()
    .merge(
        build_dmg_data_grouped[["pcode", "Totally"]]
        .groupby("pcode")
        .sum()
        .reset_index(),
        left_on="ADM3_PCODE",
        right_on="pcode",
        how="left",
    )
)
test_df["Diff"] = test_df["Totally_Damaged_bygrid"] - test_df["Totally"]
test_df["Diff"].describe()

count   1263.00000
mean      -0.00004
std        0.00141
min       -0.04138
25%       -0.00000
50%        0.00000
75%        0.00000
max        0.00795
Name: Diff, dtype: float64

A small difference between the number of totally damaged values. 
This can be explained by the weighting using the area overlap.
There may be some rounding when computing the area.

In [45]:
# computing percentage damage
# merging with building damage data
merged_perc_damage_df = merged_df_points.merge(
    merged_total_damage_df, on="id", how="right", suffixes=(None, "_y")
)
merged_perc_damage_df[
    [
        "id",
        "Centroid",
        "ADM3_PCODE",
        "ADM3_EN",
        "typhoon",
        "Year",
        "Municipality Completeness",
        "Totally",
        "numbuildings",
        "Totally_Damaged_bygrid",
    ]
].sort_values(["Totally_Damaged_bygrid"], ascending=False)

,id,Centroid,ADM3_PCODE,ADM3_EN,typhoon,Year,Municipality Completeness,Totally,numbuildings,Totally_Damaged_bygrid
19236,15273,123.4E_13.6N,PH051725000,Ocampo,Durian,2006.00000,0.73242,25951.00000,381.00000,19007.06995
9136,16299,124.0E_11.2N,PH072221000,Daanbantayan,Haiyan,2013.00000,0.67517,13660.00000,13825.00000,9222.79095
19836,17969,125.0E_11.2N,PH083739000,Palo,Haiyan,2013.00000,0.51789,13481.00000,48718.00000,6981.66883
3069,15798,123.7E_11.2N,PH072209000,Bantayan,Haiyan,2013.00000,0.59869,10533.00000,10951.00000,6305.98698
27737,17970,125.0E_11.1N,PH083748000,Tanauan,Haiyan,2013.00000,0.90848,6670.00000,10487.00000,6059.56721
...,...,...,...,...,...,...,...,...,...,...
29505,14815,123.1E_9.3N,PH074625000,Zamboanguita,NaN,NaN,0.00045,NaN,39.00000,NaN
29506,14816,123.1E_9.2N,PH074625000,Zamboanguita,NaN,NaN,0.26854,NaN,111.00000,NaN
29507,14817,123.1E_9.1N,PH074625000,Zamboanguita,NaN,NaN,0.09097,NaN,416.00000,NaN
29508,14983,123.2E_9.2N,PH074625000,Zamboanguita,NaN,NaN,0.31341,NaN,127.00000,NaN


We changed the approach to use number of buildings in a municipality instead.
Since we are using HOTOSM data, which is differently sourced from the damage data,
some municipalities and grids have more damaged buildings than total number of buildings.
TODO: Find new building dataset that is more accurate.

In [46]:
merged_perc_damage_dfout = (
    merged_perc_damage_df[
        [
            "id",
            "Centroid",
            "ADM3_PCODE",
            "ADM3_EN",
            "typhoon",
            "Year",
            "numbuildings",
            "Totally_Damaged_bygrid",
        ]
    ]
    .groupby(["id", "Centroid", "typhoon", "Year"])
    .sum(numeric_only=True)
    .reset_index()
)
# computing the percentage damage
merged_perc_damage_dfout["Totally_Damaged_Perc_bygrid"] = (
    merged_perc_damage_dfout["Totally_Damaged_bygrid"]
    / merged_perc_damage_dfout["numbuildings"]
)
merged_perc_damage_dfout.sort_values(
    ["Totally_Damaged_Perc_bygrid"], ascending=False
)

,id,Centroid,typhoon,Year,numbuildings,Totally_Damaged_bygrid,Totally_Damaged_Perc_bygrid
12985,20007,126.2E_7.8N,Bopha,2012.00000,8.00000,2280.68599,285.08575
12267,18468,125.3E_11.4N,Fengshen,2008.00000,4.00000,806.32677,201.58169
12992,20009,126.2E_7.6N,Bopha,2012.00000,12.00000,1759.18131,146.59844
12269,18468,125.3E_11.4N,Haiyan,2013.00000,4.00000,573.66872,143.41718
4988,11772,121.3E_13.0N,Melor,2015.00000,24.00000,2916.54911,121.52288
...,...,...,...,...,...,...,...
2732,11048,120.9E_18.6N,VAMCO,2020.00000,1334.00000,0.00000,0.00000
2726,11048,120.9E_18.6N,Kalmaegi,2014.00000,1334.00000,0.00000,0.00000
2717,10940,120.8E_12.7N,NAKRI,2019.00000,28.00000,0.00000,0.00000
2711,10939,120.8E_12.8N,NAKRI,2019.00000,157.00000,0.00000,0.00000


In [47]:
merged_perc_damage_dfout["Totally_Damaged_bygrid"].sum()

1645927.7189275506

In [ ]:
# writing output to CSV file
# to write to csv file, group first by grid centroid
merged_perc_damage_dfout.to_csv(
    output_dir / "building_damage_bygrid_using_area.csv", index=False
)

## Using Number of Buildings

Merging all dataframes, one with number of buildings in municipality, 
one with number of damaged buildings in municipality and 
the last with weights for each grid and municipality.

In [103]:
fji_bld_all_merged_df = fji_build_municip.merge(
    build_dmg_data_grouped,
    left_on="ADM3_PCODE",
    right_on="pcode",
    how="left",
    suffixes=(None, "_y"),
).merge(phl_build_weights, on="ADM3_PCODE", how="left", suffixes=(None, "_y"))

In [106]:
fji_bld_all_merged_df[fji_bld_all_merged_df["ADM3_PCODE"] == SUVA_PCODE]

,Unnamed: 0,ADM3_PCODE,numbuildings,typhoon,pcode,Totally,Unnamed: 0_y,id,Centroid,weight
9154,78,FJ41204,18980,Yasa,FJ41204,159.60355,589,1188,178.24E_-18.09N,0.00279
9155,78,FJ41204,18980,Yasa,FJ41204,159.60355,590,1189,178.24E_-18.19N,0.00116
9156,78,FJ41204,18980,Yasa,FJ41204,159.60355,591,1274,178.34E_-17.99N,0.00000
9157,78,FJ41204,18980,Yasa,FJ41204,159.60355,592,1275,178.34E_-18.09N,0.09257
9158,78,FJ41204,18980,Yasa,FJ41204,159.60355,593,1276,178.34E_-18.19N,0.00111
...,...,...,...,...,...,...,...,...,...,...
9277,78,FJ41204,18980,Tomas,FJ41204,37.94405,592,1275,178.34E_-18.09N,0.09257
9278,78,FJ41204,18980,Tomas,FJ41204,37.94405,593,1276,178.34E_-18.19N,0.00111
9279,78,FJ41204,18980,Tomas,FJ41204,37.94405,594,1361,178.44E_-17.99N,0.00000
9280,78,FJ41204,18980,Tomas,FJ41204,37.94405,595,1362,178.44E_-18.09N,0.74889


In [107]:
phl_build_weights.groupby("ADM3_PCODE")["weight"].sum().describe()

count   86.00000
mean     0.89535
std      0.30790
min      0.00000
25%      1.00000
50%      1.00000
75%      1.00000
max      1.00000
Name: weight, dtype: float64

In [115]:
# removed "Year" from groupby
fji_bld_all_merged_df.groupby(["pcode", "typhoon"]).first()["Totally"].sum()

47062.0

In [116]:
# removed "Year" from groupby
fji_bld_all_merged_df.groupby(["pcode", "typhoon"]).first()[
    "numbuildings"
].sum()

4073712

In [117]:
fji_build_municip["numbuildings"].sum()

254607

In [118]:
fji_bld_all_merged_df["numbuildings_bygrid"] = (
    fji_bld_all_merged_df["weight"] * fji_bld_all_merged_df["numbuildings"]
)
fji_bld_all_merged_df["Totally_Damaged_bygrid"] = (
    fji_bld_all_merged_df["weight"] * fji_bld_all_merged_df["Totally"]
)
fji_bld_all_merged_df["Totally_Damaged_bygrid"] = fji_bld_all_merged_df[
    "Totally_Damaged_bygrid"
].fillna(0)
fji_bld_all_merged_df[
    [
        "id",
        "Centroid",
        "ADM3_PCODE",
        "typhoon",
        # "Year",
        "weight",
        "Totally",
        "numbuildings",
        "numbuildings_bygrid",
        "Totally_Damaged_bygrid",
    ]
].sort_values(["Totally_Damaged_bygrid"], ascending=False)

,id,Centroid,ADM3_PCODE,typhoon,weight,Totally,numbuildings,numbuildings_bygrid,Totally_Damaged_bygrid
9176,1362,178.44E_-18.09N,FJ41204,Winston,0.74889,2263.89542,18980,14214.00000,1695.41672
4120,487,177.44E_-17.59N,FJ20107,Winston,0.56395,2515.21405,21087,11892.00000,1418.45333
8323,1362,178.44E_-18.09N,FJ40903,Winston,0.41888,2951.17494,24742,10364.00000,1236.19663
8326,1449,178.54E_-18.09N,FJ40903,Winston,0.38433,2951.17494,24742,9509.00000,1134.21399
9713,1448,178.54E_-17.99N,FJ41401,Winston,0.75108,1322.55387,11088,8328.00000,993.34673
...,...,...,...,...,...,...,...,...,...
6430,1372,178.44E_-19.09N,FJ30403,Ami,0.00000,13.85331,1325,0.00000,0.00000
6431,1455,178.54E_-18.69N,FJ30403,Ami,0.00000,13.85331,1325,0.00000,0.00000
6434,1283,178.34E_-18.89N,FJ30403,Oscar,0.00000,12.25565,1325,0.00000,0.00000
6438,1372,178.44E_-19.09N,FJ30403,Oscar,0.00000,12.25565,1325,0.00000,0.00000


In [119]:
fji_bld_all_merged_df[fji_bld_all_merged_df["ADM3_PCODE"] == SUVA_PCODE][
    [
        "id",
        "Centroid",
        "ADM3_PCODE",
        "typhoon",
        # "Year",
        "weight",
        "Totally",
        "numbuildings",
        "numbuildings_bygrid",
        "Totally_Damaged_bygrid",
    ]
]

,id,Centroid,ADM3_PCODE,typhoon,weight,Totally,numbuildings,numbuildings_bygrid,Totally_Damaged_bygrid
9154,1188,178.24E_-18.09N,FJ41204,Yasa,0.00279,159.60355,18980,53.00000,0.44568
9155,1189,178.24E_-18.19N,FJ41204,Yasa,0.00116,159.60355,18980,22.00000,0.18500
9156,1274,178.34E_-17.99N,FJ41204,Yasa,0.00000,159.60355,18980,0.00000,0.00000
9157,1275,178.34E_-18.09N,FJ41204,Yasa,0.09257,159.60355,18980,1757.00000,14.77468
9158,1276,178.34E_-18.19N,FJ41204,Yasa,0.00111,159.60355,18980,21.00000,0.17659
...,...,...,...,...,...,...,...,...,...
9277,1275,178.34E_-18.09N,FJ41204,Tomas,0.09257,37.94405,18980,1757.00000,3.51252
9278,1276,178.34E_-18.19N,FJ41204,Tomas,0.00111,37.94405,18980,21.00000,0.04198
9279,1361,178.44E_-17.99N,FJ41204,Tomas,0.00000,37.94405,18980,0.00000,0.00000
9280,1362,178.44E_-18.09N,FJ41204,Tomas,0.74889,37.94405,18980,14214.00000,28.41605


In [120]:
fji_bld_all_merged_df["Totally_Damaged_bygrid"].sum()

47061.999999999985

In [121]:
fji_bld_all_merged_df["numbuildings_bygrid"].sum()

4073711.9999999986

In [124]:
fji_bld_all_merged_dfout = fji_bld_all_merged_df[
    [
        "id",
        "Centroid",
        "typhoon",
        # "Year",
        "weight",
        "numbuildings_bygrid",
        "Totally_Damaged_bygrid",
    ]
]
# removed "Year" from groupby
fji_bld_all_merged_dfout = (
    fji_bld_all_merged_dfout.groupby(["id", "Centroid", "typhoon"])
    .sum()
    .reset_index()
)

fji_bld_all_merged_dfout["Totally_Damaged_Perc_bygrid"] = (
    fji_bld_all_merged_dfout["Totally_Damaged_bygrid"]
    / fji_bld_all_merged_dfout["numbuildings_bygrid"]
)
fji_bld_all_merged_dfout.sort_values(
    ["Totally_Damaged_Perc_bygrid"], ascending=False
)

,id,Centroid,typhoon,weight,numbuildings_bygrid,Totally_Damaged_bygrid,Totally_Damaged_Perc_bygrid
4078,2302,179.54E_-16.39N,Winston,0.09424,845.00000,100.78986,0.11928
2830,1533,178.64E_-17.79N,Winston,0.01929,107.00000,12.76274,0.11928
1230,920,177.94E_-17.39N,Winston,0.02764,146.00000,17.41458,0.11928
3198,1782,178.94E_-16.59N,Winston,0.22006,1178.00000,140.50942,0.11928
3918,2216,179.44E_-16.49N,Winston,0.25249,2546.00000,303.68165,0.11928
...,...,...,...,...,...,...,...
5883,4156,181.64E_-19.09N,Tia,0.00000,0.00000,0.00000,NaN
5884,4156,181.64E_-19.09N,Tomas,0.00000,0.00000,0.00000,NaN
5885,4156,181.64E_-19.09N,Wilma,0.00000,0.00000,0.00000,NaN
5886,4156,181.64E_-19.09N,Winston,0.00000,0.00000,0.00000,NaN


In [125]:
fji_bld_all_merged_dfout["numbuildings_bygrid"].sum()

4073711.999999999

In [126]:
fji_bld_all_merged_dfout[fji_bld_all_merged_dfout["Centroid"] == "126.6E_7.3N"]

,id,Centroid,typhoon,weight,numbuildings_bygrid,Totally_Damaged_bygrid,Totally_Damaged_Perc_bygrid


In [127]:
fji_bld_all_merged_dfout.groupby(["Centroid"])[
    "numbuildings_bygrid"
].sum().reset_index()

,Centroid,numbuildings_bygrid
0,176.94E_-17.09N,1008.00000
1,176.94E_-17.19N,2384.00000
2,177.04E_-17.59N,5072.00000
3,177.14E_-17.19N,416.00000
4,177.14E_-17.29N,10528.00000
...,...,...
364,181.54E_-18.69N,4208.00000
365,181.54E_-19.09N,0.00000
366,181.54E_-19.19N,0.00000
367,181.64E_-19.09N,0.00000


In [128]:
fji_bld_all_merged_dfout["Totally_Damaged_bygrid"].sum()

47061.999999999985

In [129]:
fji_bld_all_merged_dfout["Totally_Damaged_Perc_bygrid"].describe()

count   5120.00000
mean       0.01155
std        0.02834
min        0.00000
25%        0.00022
50%        0.00130
75%        0.00862
max        0.11928
Name: Totally_Damaged_Perc_bygrid, dtype: float64

In [130]:
# writing output to CSV file
# to write to csv file, group first by grid centroid
fji_bld_all_merged_dfout.to_csv(
    output_dir / "building_damage_bygrid.csv", index=False
)